# GENERAMOS LAS INSTANCIAS:

##### Importamos las librerias a usar

In [1]:
import math, subprocess, time, random, os, shutil, sys
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
def save_instance(dataset, instance_name, b,c, m):
    with open(F"instances/{dataset}/{instance_name}.txt", "w") as f:
        print(len(c), m, file=f)
        for j in range(0,len(c)): 
            print(b[j], file=f, end=" ")
            print(c[j], file=f, end=" \n")


In [3]:
def clean_data(nombre):
    try:
        shutil.rmtree("instances/"+nombre)
    except OSError as e:
        print ("Error: %s - %s." % (e.filename, e.strerror))
    if os.path.isfile("results/result.csv"):
        df = pd.read_csv("results/result.csv")
        df =  df[df.dataset != nombre]
        df.to_csv("results/result.csv", index=False)
        

    os.makedirs("instances/"+nombre, exist_ok=False)

In [4]:
def plot(title,bf=True,bt=True,bto=True,btf=True,dp=True,l = False,x=9,y=7):
    fig = plt.figure(figsize =(x, y))
    
    if (l) :
        l = "" 
    else:
        l = "o" 
    

    df_bf = df_resultados[(df_resultados["method"]=="bf")]

    df_bt = df_resultados[(df_resultados["method"]=="bt")]

    df_bto = df_resultados[(df_resultados["method"]=="bto")]

    df_btf = df_resultados[(df_resultados["method"]=="btf")]

    df_dp = df_resultados[(df_resultados["method"]=="dp")]

    plt.plot()

    #puntos
    plt.plot(df_bf["n"],df_bf["time"],l, c="black", label="bf")
    plt.plot(df_bt["n"],df_bt["time"],l, c="blue", label="bt")
    plt.plot(df_bto["n"],df_bto["time"],l, c="green", label="bto")
    plt.plot(df_btf["n"],df_btf["time"],l, c="r", label="btf")
    plt.plot(df_dp["n"],df_dp["time"],l, c="purple", label="dp")

    plt.legend()
    plt.grid()



    plt.title(title)
    plt.xlabel('n') 
    plt.ylabel('Clocks')  
    plt.show()

#### En esta seccion nos centraremos en correr los experimentos con las instancias generadass mas arriba

#### Funciones a usar:

In [11]:
def run_program(method, test_file):
    #corre el programa con el metodo indicado en el test file indicado y devuelve el tiempo tomado en ejecutar
    #leemos la instancia
    
    start = time.time() #comenzamos a medir el timepo
    proc = subprocess.Popen(["../src/tp1",test_file,method], stderr=subprocess.PIPE,stdout = subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True) #lanzamos proceso
    #
    
    exit_code = proc.wait()#esperamos a que el proceso termien
    
    end = time.time() #terminamos de medir el tiempo
    
    #mandamos error de ser necesario 
    if exit_code != 0: raise RuntimeError(F"Hubo un error en la experimentacion para el algoritmo: {method} con la instancia {test_file}.")
    
    return end-start #devolvemos el timepo
    

In [12]:
def run_experiments():
    name = "exp"
    #recibe un nombre para darle al archivo de salida y lo que hace es correr todos los experimentos que se encuentren en la lista llamada experiments que debe ser generada con anterioridad
    columns = ["dataset","exp_name","n", "m", "method", "time"]
    rows = []
    j = 1
    T =10 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
    explen = str(len(experiments))
    
    
    for exp in tqdm(experiments, "Completed experiments"):
#         print(exp)
        times = []
        for i in range(0, T):
            times.append(run_program(exp[3], exp[4]))
        rows.append([exp[0], name+str(j), exp[1], exp[2],exp[3], np.mean(times)])
        j +=1
    result = pd.DataFrame(rows, columns=columns)
    
    if os.path.isfile("results/result.csv"):
        df = pd.read_csv("results/result.csv")
        df = pd.concat([df, result], ignore_index = True, axis = 0)
    else:
        df = result
    
    
    df.to_csv("results/result.csv", index=False, header=True)

## Corrida de experimentos

#### Leemos los indices de las instancias generadas anteriormente 

## Experimento 1

In [179]:
def exp1(k=41):
    
    filas_indice = []
    nombre = "peor_btf"
    if (k!=41): nombre = "peor_btf_bf"


#     clean_data(nombre)
    
    
    for n in tqdm(range(10, k,1), "Generating instances"):
        
        #damos valores a n,w,b,c
        m = n+2
        c = [1 for i in range(0,n)]
        b = [(n-i) for i in range(0,n)]
        b[-1] = 1000
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)

In [135]:
clean_data("peor_btf")

In [138]:
exp1()

Generating instances:   0%|          | 0/31 [00:00<?, ?it/s]

In [139]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos

instance = "peor_btf"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])

run_experiments()


Completed experiments:   0%|          | 0/124 [00:00<?, ?it/s]

## Experimento 2

In [177]:
def exp2(k=41): #este experimento sera utilizado para la hipotesis 1

    filas_indice = []
    nombre = "peor_bto"
    if (k!=41): nombre = "peor_bto_bf"


#     clean_data(nombre)
    
    
    for n in tqdm(range(10, k,1), "Generating instances"):
        
        #damos valores a n,w,b,c
        m = 5
        c = [1 for i in range(0,n)]
        b = [0 for i in range(0,n)]
        b[-1] = n**2
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)

In [ ]:
clean_data("peor_bto")

In [ ]:
exp2()

In [25]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos

instance = "peor_bto"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])

run_experiments()


Completed experiments:   0%|          | 0/124 [00:00<?, ?it/s]

## Experimento 3

In [124]:
def exp3(): #este experimento sera usado para la hipotesis 2
    filas_indice = []
    
    nombre = "m_1"
        
#     clean_data(nombre)
    for n in tqdm(range(2, 10001,1000), "Generating instances"):
       
        
        #damos valores a n,w,b,c
        m= 1000
        b = [1 for i in range(0,n)]
        c = [1 for i in range(0,n)]
        
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)
    
    #######################
    filas_indice = []
    
    nombre = "m_2"
#     clean_data(nombre)
    for n in tqdm(range(2,10001,1000), "Generating instances"):
       
        
        #damos valores a n,w,b,c
        m= 2000
        b = [1 for i in range(0,n)]
        c = [1 for i in range(0,n)]
        
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)
    
    ##########################
    filas_indice = []
    nombre = "m_3"
#     clean_data(nombre)
    for n in tqdm(range(2, 10001,1000), "Generating instances"):
       
        
        #damos valores a n,w,b,c
        m= 3000
        b = [1 for i in range(0,n)]
        c = [1 for i in range(0,n)]
        
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)
    
    ##########################
    filas_indice = []
    nombre = "m_4"
#     clean_data(nombre)
    for n in tqdm(range(2, 10001,1000), "Generating instances"):
       
        
        #damos valores a n,w,b,c
        m= 4000
        b = [1 for i in range(0,n)]
        c = [1 for i in range(0,n)]
        
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)

In [125]:
clean_data("m_1")
clean_data("m_2")
clean_data("m_3")
clean_data("m_4")

In [126]:
exp3()

Generating instances:   0%|          | 0/10 [00:00<?, ?it/s]

Generating instances:   0%|          | 0/10 [00:00<?, ?it/s]

Generating instances:   0%|          | 0/10 [00:00<?, ?it/s]

Generating instances:   0%|          | 0/10 [00:00<?, ?it/s]

In [127]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos


instance = "m_1"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])


instance = "m_2"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])

instance = "m_3"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])
    
instance = "m_4"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])
    
#corremos los experimentos
run_experiments()


Completed experiments:   0%|          | 0/40 [00:00<?, ?it/s]

## Experimento 4

In [114]:
def exp4(): 
    filas_indice = []
    
    nombre = "n_1"
    
#     clean_data(nombre)
    n = 20
    for m in tqdm(range(2, 100001,1000), "Generating instances"):
       
        
        #damos valores a n,w,b,c
        
        b = [1 for i in range(0,n)]
        c = [1 for i in range(0,n)]
        
        save_instance(nombre, nombre+F"-{m}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{m}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)
    
    #######################
    
    filas_indice = []
    nombre = "n_2"
    n= 25
#     clean_data(nombre)
    for m in tqdm(range(2,100001,1000), "Generating instances"):
       
        b = [1 for i in range(0,n)]
        c = [1 for i in range(0,n)]
        
        save_instance(nombre, nombre+F"-{m}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{m}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)
    
    ########################
    filas_indice = []
    nombre = "n_3"
#     clean_data(nombre)
    n = 30
    for m in tqdm(range(2, 100001,1000), "Generating instances"):
       
        
        
        b = [1 for i in range(0,n)]
        c = [1 for i in range(0,n)]
        
        save_instance(nombre, nombre+F"-{m}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{m}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)
    
    ########################
    filas_indice = []
    nombre = "n_4"
#     clean_data(nombre)
    n=35
    for m in tqdm(range(2, 100001,1000), "Generating instances"):
       
        
        b = [1 for i in range(0,n)]
        c = [1 for i in range(0,n)]
        
        save_instance(nombre, nombre+F"-{m}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{m}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)

In [115]:
clean_data("n_1")
clean_data("n_2")
clean_data("n_3")
clean_data("n_4")

In [116]:
exp4()

Generating instances:   0%|          | 0/100 [00:00<?, ?it/s]

Generating instances:   0%|          | 0/100 [00:00<?, ?it/s]

Generating instances:   0%|          | 0/100 [00:00<?, ?it/s]

Generating instances:   0%|          | 0/100 [00:00<?, ?it/s]

In [117]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos


instance = "n_1"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])


instance = "n_2"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])

instance = "n_3"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])
    
instance = "n_4"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "dp", row_k["file"]])
    
#corremos los experimentos
run_experiments()


Completed experiments:   0%|          | 0/400 [00:00<?, ?it/s]

## Experimento particular bt

In [170]:
def exp5(): 
    filas_indice = []
    
    nombre = "particular_bt"
        
#     clean_data(nombre)
    for n in tqdm(range(10, 41,1), "Generating instances"):
       
        
        #damos valores a n,w,b,c
        m = int(n/4)
        c = [1 for i in range(0,n)]
        b = [i**2 for i in range(0,n)]
        
#         m= np.random.randint(100,1000)
#         b = np.random.randint(100, size=n)
#         c = np.random.randint(100, size=n)
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)



In [171]:
clean_data("particular_bt")

In [172]:
exp5()

Generating instances:   0%|          | 0/31 [00:00<?, ?it/s]

In [173]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos

instance = "particular_bt"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "btf", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bt", row_k["file"]])
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bto", row_k["file"]])
#     experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])

run_experiments()


Completed experiments:   0%|          | 0/93 [00:00<?, ?it/s]

## EXPERIMENTO FB

Realizamos la experimentacion con fuerza bruta aparte debido al tiempo de computo requerido para realizarla

In [183]:
def exp6(): 
    filas_indice = []
    
    nombre = "random_bf"
        
#     clean_data(nombre)
    for n in tqdm(range(10, 31,1), "Generating instances"):
       
        
        #damos valores a n,w,b,c
        m = np.random.randint(0, 1000)
        c = [np.random.randint(0, 100) for i in range(0,n)]
        b = [np.random.randint(0, 100) for i in range(0,n)]
        
        save_instance(nombre, nombre+F"-{n}",b,c, m)
        filas_indice.append([nombre, nombre + f"-{n}", n, m, "instances/"+nombre+"/"+nombre+f"-{n}.txt"])
    pd.DataFrame(filas_indice, columns=["dataset", "instance", "n", "m", "file"]).to_csv("instances/"+nombre+"/indice.csv", index=False, header=True)



In [184]:
clean_data("peor_bto_bf")
clean_data("peor_btf_bf")
clean_data("random_bf")

In [185]:
exp1(31)
exp2(31)
exp6()

Generating instances:   0%|          | 0/21 [00:00<?, ?it/s]

Generating instances:   0%|          | 0/21 [00:00<?, ?it/s]

Generating instances:   0%|          | 0/21 [00:00<?, ?it/s]

In [187]:
experiments = [] #vaciamos lista para llenarla con los datos que queremos


instance = "peor_bto_bf"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])


instance = "peor_btf_bf"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])

instance = "random_bf"
df_data = pd.read_csv("instances/"+instance+"/indice.csv")
for k in range(0, len(df_data)):
    row_k = df_data.iloc[k]
    experiments.append([row_k["dataset"], row_k["n"], row_k["m"], "bf", row_k["file"]])
    
#corremos los experimentos
run_experiments()


Completed experiments:   0%|          | 0/63 [00:00<?, ?it/s]